# KnapSack

Experimento de resolução do problema da mochila usando algoritmos genéticos no framework DEAP

## Dependências

In [167]:
import random
import time
import pandas as pd
from deap import creator, base, tools, algorithms

## Variáveis iniciais

In [168]:
# Lista de itens: ID, Nome do item, Valor (R$), Peso (kg)
items = [
    (1, "Barraca", 150.0, 3.5),
    (2, "Saco de dormir", 100.0, 2.0),
    (3, "Isolante térmico", 50.0, 0.5),
    (4, "Colchão inflável", 80.0, 1.0),
    (5, "Lanterna", 30.0, 0.2),
    (6, "Kit de primeiros socorros", 20.0, 0.5),
    (7, "Repelente de insetos", 15.0, 0.1),
    (8, "Protetor solar", 20.0, 0.2),
    (9, "Canivete", 10.0, 0.1),
    (10, "Mapa e bússola", 25.0, 0.3),
    (11, "Garrafa de água", 15.0, 1.8),
    (12, "Filtro de água", 50.0, 0.5),
    (13, "Comida (ração liofilizada)", 50.0, 3.0),
    (14, "Fogão de camping", 70.0, 1.5),
    (15, "Botijão de gás", 30.0, 1.2),
    (16, "Prato, talheres e caneca", 20.0, 0.5),
    (17, "Roupas (conjunto)", 80.0, 1.5),
    (18, "Calçados (botas)", 120.0, 2.0),
    (19, "Toalha", 20.0, 0.5),
    (20, "Kit de higiene pessoal", 30.0, 0.5)
]

# Capacidade máxima da mochila em kg
max_weight = 15.0

## Parâmetros do algoritmo genético

### População inicial

In [169]:
population_size = 300

Número de indivíduos na população inicial

**Considerações**

Diversidade Inicial: Um tamanho maior da população aumenta a probabilidade de <mark>exploração</mark> uma parte mais ampla do espaço de busca no início, mas populações maiores podem retardar a convergência.

Convergência: Populações menores tendem a convergir mais rapidamente, favorecendo a <mark>exploitação</mark> de regiões específicas, mas podem ficar presas em mínimos locais.

Custo Computacional: Populações maiores aumentam o número de avaliações de fitness por geração, o que pode aumentar o tempo de execução.

### Probabilidade individual de mutação

In [170]:
individual_probability = 0.05

Probabilidade de mutação de cada gene em um indivíduo (bit flip)

**Considerações**

Valores baixos (e.g., 0.01): Introduzem mutação esporádica, preservando a maior parte dos genes originais. É útil para pequenas alterações no indivíduo.

Valores altos (e.g., 0.5): Introduzem mutação mais intensa, alterando mais genes. Pode ajudar a escapar de mínimos locais, mas corre o risco de desestabilizar soluções já boas.

### Tamanho do torneio

In [171]:
tournament_size = 3

Tamanho do torneio, ou número de individuos selecionados para reprodução

**Brevemente, como Funciona a Seleção por Torneio:**

Subconjunto Aleatório: Para cada seleção, o algoritmo escolhe aleatoriamente um grupo de indivíduos da população. O número de indivíduos nesse grupo é definido por *tournament_size*.

Competição: O indivíduo com o melhor valor de fitness dentro desse grupo é selecionado.

Repetição: O processo é repetido até que o número desejado de indivíduos seja selecionado.

**Considerações**

Valores pequenos (e.g., tournsize=2):

A seleção se torna mais aleatória. Há maior diversidade na próxima geração, pois indivíduos com fitness baixos ainda têm chances razoáveis de serem selecionados. Pode ser útil no início do algoritmo, onde a <mark>exploração</mark> é mais importante.

Valores grandes (e.g., tournsize=10):

A seleção se torna mais elitista, favorecendo indivíduos com fitness altos. Isso acelera a convergência, mas pode levar o algoritmo a mínimos locais se a diversidade for insuficiente. Pode seu útil após achar as áreas mais favoráveis do problema pois favorece a <mark>exploitação</mark> do local.

Valor igual ao tamanho da população:

A seleção se torna totalmente determinística, pois sempre escolherá o indivíduo com o melhor fitness na população.

### Número de Gerações

In [172]:
num_generations = 40

É o número de iterações que o algoritmo genético realizará, a cada geração, a população de indivíduos evolui com base em operadores como crossover (recombinação de pais) e mutação.

**Considerações**

Mais gerações: Pode levar a um processo de evolução mais demorado, mas também oferece mais oportunidades para o algoritmo encontrar uma solução melhor.

Menos gerações: Pode acelerar a execução, mas também aumenta o risco de o algoritmo não ter tempo suficiente para explorar adequadamente o espaço de soluções.

### Taxa de Crossover

In [173]:
crossover_probability = 0.5

Define a probabilidade de que dois indivíduos (pais) se cruzem para gerar um ou mais filhos. No crossover, partes dos genes de dois indivíduos são trocadas para criar novos indivíduos.

**Considerações**

Taxa de Crossover alta (e.g., 0.9): Mais cruzamentos entre indivíduos, promovendo mais exploração e diversidade. Pode acelerar a busca por boas soluções, mas também pode resultar em soluções mais diversificadas e não tão refinadas.

Taxa de Crossover baixa (e.g., 0.1): Menos recombinação entre indivíduos, promovendo mais exploração local das soluções, o que pode ser bom para refinar uma solução existente, mas pode limitar a diversidade.

### Taxa de Mutação

In [174]:
mutation_probability = 0.2

Define a probabilidade de que um indivíduo sofra uma mutação em seus genes e introduz novas variações aleatórias no processo evolutivo.

**Considerações**

Taxa de Mutação alta (e.g., 0.5): Mais mutações, o que pode resultar em maior <mark>exploração</mark> do espaço de busca. No entanto, uma taxa muito alta pode fazer o algoritmo perder boas soluções já encontradas.

Taxa de Mutação baixa (e.g., 0.01): Menos mutações, favorecendo a <mark>exploitação</mark> local de soluções e uma busca mais direcionada, mas com risco de convergência prematura (ficar preso em mínimos locais).

### Melhores indivíduos

In [175]:
best_individuals = 5

Número de indivíduos selecionados ao final

## Configuração do DEAP

### Função de avaliação

In [176]:
def evalKnapsack(individual):
    total_value = 0.0  # Valor total da mochila
    total_weight = 0.0  # Peso total da mochila

    for i, selected in enumerate(individual):
        if selected:  # Se o item foi selecionado
            total_value += items[i][2]  # Soma o valor do item ao total
            total_weight += items[i][3]  # Soma o peso do item ao total

    # Penaliza soluções que ultrapassam o peso máximo
    if total_weight > max_weight:
        return -1,  # Penalidade
    return total_value, # Retorna o valor total como fitness para soluções válidas

### Montagem do algoritmo

In [177]:
# Cria um tipo de "fitness" para maximizar um único valor (peso positivo indica maximização).
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

# Define um indivíduo como uma lista que possui um fitness do tipo "FitnessMax".
creator.create("Individual", list, fitness=creator.FitnessMax)

# Caixa de ferramentas para registrar funções que geram e operam sobre os indivíduos.
toolbox = base.Toolbox()

# Representação binária dos itens (0 ou 1 para incluir ou não incluir)
toolbox.register("attr_bool", random.randint, 0, 1)

# Registra um indivíduo como uma lista de genes de comprimento igual ao número de itens.
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(items))

# Registra uma população como uma lista de indivíduos.
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Registra a função de avaliação no toolbox.
toolbox.register("evaluate", evalKnapsack)

# Registra o operador de crossover de dois pontos no toolbox.
toolbox.register("mate", tools.cxTwoPoint)

# Registra o operador de mutação.
toolbox.register("mutate", tools.mutFlipBit, indpb=individual_probability)

# Registra o operador de seleção por torneio.
toolbox.register("select", tools.selTournament, tournsize=tournament_size)

# Gera a população inicial
population = toolbox.population(n=population_size)

c:\ProgramData\Miniconda3\envs\scipy\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\ProgramData\Miniconda3\envs\scipy\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


## Execução

In [178]:
# Registra o tempo de início
start_time = time.time()

# Execução do algoritmo
for gen in range(num_generations):

    # Gera novos indivíduos (offspring) a partir da população atual aplicando crossover e mutação.
    offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_probability, mutpb=mutation_probability)

    # Calcula os valores de fitness para cada indivíduo da nova geração.
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit  # Atribui o fitness calculado aos indivíduos.

    population = toolbox.select(offspring, k=len(population))

# Seleciona n melhores indivíduos.
top_individuals = tools.selBest(population, k=best_individuals)

# Registra o tempo de término
end_time = time.time()

# Calcula o tempo total de execução
execution_time = end_time - start_time

## Resultados (Melhor indivíduo)

In [179]:
# Seleciona o melhor indivíduo.
best_individual = top_individuals[0]

# Identifica os itens selecionados no melhor indivíduo.
selected_items = [items[i] for i in range(len(best_individual)) if best_individual[i] == 1]

In [180]:
# Criação do DataFrame com os itens selecionados pelo melhor indivíduo
selected_items_data = [{
    'Nome do item': item[1],
    'Valor (R$)': item[2],
    'Peso (kg)': item[3]
} for item in selected_items]

df_selected_items = pd.DataFrame(selected_items_data)

print("Itens selecionados:")
df_selected_items.head(len(items))

Itens selecionados:


,Nome do item,Valor (R$),Peso (kg)
0,Barraca,150.0,3.5
1,Saco de dormir,100.0,2.0
2,Isolante térmico,50.0,0.5
3,Colchão inflável,80.0,1.0
4,Lanterna,30.0,0.2
5,Repelente de insetos,15.0,0.1
6,Protetor solar,20.0,0.2
7,Canivete,10.0,0.1
8,Mapa e bússola,25.0,0.3
9,Filtro de água,50.0,0.5


In [181]:
# Imprime o valor total dos itens na mochila.
print(f"Valor total: R${sum(item[2] for item in selected_items):.2f}")

Valor total: R$870.00


In [182]:
# Imprime o peso total dos itens na mochila.
print(f"Peso total: {sum(item[3] for item in selected_items):.2f}kg")

Peso total: 14.90kg


In [183]:
# Imprime o tempo de execução
print(f"Tempo de execução (evolução somente): {execution_time:.4f} segundos")

Tempo de execução (evolução somente): 0.2124 segundos
